In [3]:
import tensorflow as tf

In [42]:
trainpath = tf.io.gfile.glob(r"C:\Users\Brothers\Desktop\Python\ongoing_project\MY STUFF\Tensorflow\CreatingImageDataset\Using google tpu\data\*")

In [20]:
# creating image dataset
# as a parameter you need to give the pattern of the images location
trainDs = tf.data.Dataset.list_files(r"C:\Users\Brothers\Desktop\Python\ongoing_project\MY STUFF\Tensorflow\CreatingImageDataset\Using google tpu\data\*\*.png")
trainDs
# testDs = tf.data.Dataset.list_files("gs://kds-b901ee9b8bcbf3fc0a0b62fec6db227bad0ebfae07b071fa829973aa/seg_test/seg_test/*/*.jpg")

<DatasetV1Adapter shapes: (), types: tf.string>

In [51]:
print(trainpath[0:2])
trainpath[0].split('\\')[-1]

['C:\\Users\\Brothers\\Desktop\\Python\\ongoing_project\\MY STUFF\\Tensorflow\\CreatingImageDataset\\Using google tpu\\data\\recordA', 'C:\\Users\\Brothers\\Desktop\\Python\\ongoing_project\\MY STUFF\\Tensorflow\\CreatingImageDataset\\Using google tpu\\data\\recordB']


'recordA'

In [53]:
import numpy as np

In [57]:
trainpath = [trainpath[0], trainpath[1]]
train_label = []
for i in trainpath:
    train_label.append(i.split('\\')[-1])
classes = np.array(train_label)

'forest' == classes

In [59]:
classes

array(['recordA', 'recordB'], dtype='<U7')

# A helper function for image decoding and augumentation

In [60]:
IMG_WIDTH=128*2
IMG_HEIGHT=128*2
def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32) 
    img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT]) 
#     img = tf.image.random_flip_left_right(img)
#     img = tf.image.random_flip_up_down(img)
#     img = tf.image.random_brightness(img, 0.3)
    return img

def get_label(path):
    part_list = tf.strings.split(path, "/")
  # in the case where each class of images is in one folder
    return part_list[-2]==train_label

def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

#this is how get_label function will work()
'forest' == classes

# Creating train and test dataset

In [61]:
# num_threads = 5 
train_ds = trainDs.map(process_path)#, num_parallel_calls=num_threads)

train_ds

# train_ds = train_ds.cache()  # not recomended for large dataset
train_ds = train_ds.shuffle(100)

train_ds

num_epochs = 1
train_ds = train_ds.repeat(num_epochs)

train_ds

train_ds = train_ds.batch(128)

train_ds

train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

train_ds

<DatasetV1Adapter shapes: ((None, 256, 256, 3), (None, 2)), types: (tf.float32, tf.bool)>

# Model creating 

In [66]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

In [71]:
img_size = 128*2

# with tpu_strategy.scope():
model = Sequential([
        Conv2D(1, 2, 2,activation='relu', padding='same', input_shape=(img_size,img_size, 3)), 
#         Conv2D(64, 2, 2,activation='relu', padding='same'),
        MaxPool2D(padding='same'),

#         Dropout(0.7),

#         Conv2D(128, 2, 2,activation='relu', padding='same'), 
#         Conv2D(128, 2, 2,activation='relu', padding='same'),
#         MaxPool2D(padding='same'),

#         Dropout(0.7),

#         Conv2D(512, 2, 2,activation='relu', padding='same'), 
#         Conv2D(512, 2, 2,activation='relu', padding='same'),
#         MaxPool2D(padding='same'),

    Flatten(),

    Dropout(0.7),

    Dense(1028, activation='relu'),
    Dropout(0.7),
    Dense(6, activation='softmax')
])

In [72]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [73]:
# model.summary()

In [74]:
history = model.fit(train_ds, epochs=2)

Epoch 1/2
      1/Unknown - 1s 1s/step

InvalidArgumentError:   slice index -1 of dimension 0 out of bounds.
	 [[node strided_slice (defined at C:\Users\Brothers\anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]
	 [[IteratorGetNext]] [Op:__inference_distributed_function_1146]

Function call stack:
distributed_function -> distributed_function
